In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [2]:
data = pd.read_csv("/kaggle/input/quora-question-pairs/train.csv.zip", nrows=1000)
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=94cd1e9ce74ca2fcf5a76f29483b2c7c73339fd041f8022e0fb88364e1d293ec
  Stored in directory: /root/.cache/pip

In [4]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Create Train Set

def create_train_set(X):
    train_set = []
    for i,row in X.iterrows():
        train_set.append(InputExample(texts=[row['question1'],row['question2']],label=float(row['is_duplicate'])))
    return train_set

In [6]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(data.drop(['id','qid1','qid2'],axis=1),data.drop(['is_duplicate'],axis=1))

In [7]:
X = create_train_set(train_X)

In [8]:
train_dataloader  = DataLoader(X,shuffle=True)
train_loss = losses.CosineSimilarityLoss(model)
model.fit(train_objectives=[(train_dataloader,train_loss)],epochs=1)

Epoch: 100%|██████████| 1/1 [03:16<00:00, 196.69s/it]


In [9]:
type(test_X)

pandas.core.frame.DataFrame

In [11]:
test_X.head()

,question1,question2,is_duplicate
621,What are some unusual aspects about politics a...,What are some unusual aspects about politics a...,0
758,Cochin to London etihad is the change over tim...,What is difference between China and the Unite...,0
753,What do think of when you hear the word elect?,"What do you think when you hear the word, ""Jap...",0
536,What is an interesting fact that you know and ...,What is the most interesting fact that most pe...,1
958,How comfortably can I live in Washington DC on...,"Can I live comfortably in DC on $80,000 - $114...",1


In [16]:
#Now Evaluating using test set
from sentence_transformers import util
preds = []
for i,row in test_X.iterrows():
    #print(row)
    sent1 = row['question1']
    sent2 = row['question2']
    enc_sent1 = model.encode(sent1)
    enc_sent2 = model.encode(sent2)
    preds.append(util.cos_sim(enc_sent1,enc_sent2))
preds

[tensor([[0.7887]]),
 tensor([[0.1838]]),
 tensor([[0.3834]]),
 tensor([[0.6664]]),
 tensor([[0.9261]]),
 tensor([[0.5849]]),
 tensor([[0.9972]]),
 tensor([[0.6937]]),
 tensor([[0.8339]]),
 tensor([[0.8240]]),
 tensor([[0.6938]]),
 tensor([[0.4488]]),
 tensor([[0.8300]]),
 tensor([[0.3899]]),
 tensor([[0.8574]]),
 tensor([[0.5591]]),
 tensor([[0.7233]]),
 tensor([[0.6356]]),
 tensor([[0.7930]]),
 tensor([[0.8219]]),
 tensor([[0.7294]]),
 tensor([[0.3742]]),
 tensor([[0.7807]]),
 tensor([[0.8317]]),
 tensor([[0.9389]]),
 tensor([[0.4805]]),
 tensor([[0.2208]]),
 tensor([[0.5424]]),
 tensor([[0.7966]]),
 tensor([[0.9528]]),
 tensor([[0.5031]]),
 tensor([[0.8671]]),
 tensor([[0.7963]]),
 tensor([[0.7551]]),
 tensor([[0.3704]]),
 tensor([[0.9814]]),
 tensor([[0.4967]]),
 tensor([[0.0942]]),
 tensor([[0.0282]]),
 tensor([[0.7657]]),
 tensor([[0.7269]]),
 tensor([[0.9617]]),
 tensor([[0.7867]]),
 tensor([[0.3437]]),
 tensor([[0.6089]]),
 tensor([[0.6653]]),
 tensor([[0.8404]]),
 tensor([[0.8

In [17]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [21]:
test_X.columns

Index(['question1', 'question2', 'is_duplicate'], dtype='object')

In [31]:
threshold = 0.5
preds_encoded = np.where(np.array(preds)>0.5,1,0)
preds_encoded

/tmp/ipykernel_15/679214653.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  preds_encoded = np.where(np.array(preds)>0.5,1,0)
/tmp/ipykernel_15/679214653.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  preds_encoded = np.where(np.array(preds)>0.5,1,0)


array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1])

In [32]:
print('Precision: ',precision_score(test_X['is_duplicate'].tolist(),preds_encoded))

Precision:  0.4264705882352941


In [34]:
print('Recall: ',recall_score(test_X['is_duplicate'].tolist(),preds_encoded))

Recall:  0.9886363636363636


In [35]:
print('Accuracy: ',accuracy_score(test_X['is_duplicate'].tolist(),preds_encoded))

Accuracy:  0.528


In [36]:
from sklearn.metrics import f1_score
print('F1 Score: ',f1_score(test_X['is_duplicate'].tolist(),preds_encoded))

F1 Score:  0.5958904109589042
